### Portail meteo.data.gouv.fr - Téléchargement-affichage-Extraction des données HORAIRES LATEST de Météo-France (RR-T-Vent)
Une connexion internet est nécessaire pour le téléchargement automatique des archives de données.

1) Téléchargement des fichiers HORAIRES et décompression automatique, pour différents postes (si besoin dans plusieurs départements mais au prix d'une longue durée de lecture)
2) Tracé du graphique chronologique HORAIRE pour le paramètre Précipitations RR des postes choisis par l'utilisateur
3) Tracé du graphique chronologique de l'agrégation QUOTIDIENNE
4) Sauvegarde d'un fichier excel rassemblant:
    - la comparaison des paramètres QUOTIDIENS & MENSUELS pour les postes et la période choisis, ainsi que les graphiques pour les précipitations uniquement
    - NB: Les données intégrales HORAIRES ne sont pas sauvegardées pour cause de poids excessif (tous paramètres de la période "Latest" pour tous les postes des départements concernés).

data : https://meteo.data.gouv.fr/ (6 min, horaire, quotidien, mensuel)<br>
Fiche PDF des postes : https://www.data.gouv.fr/fr/datasets/r/bee4b0c7-260a-40fe-b463-ed5631d6dc39 (paramètres et périodes de mesure)<br>
Fichier CSV descriptif champs: https://www.data.gouv.fr/fr/datasets/r/5d0f9af9-149b-463a-9472-445dafb698d9

Utilisez mon autre script pour visualiser la carte des postes météorologiques dont la liste est fournie par Météo-France sous forme de fichier JSON https://meteo.data.gouv.fr/https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922

NB: 
- Les données LATEST correspondent aux DERNIERS FICHIERS mis à jour quotidiennement, et qui vont du mois de janvier de l'année précédente à la veille du jour en cours même partielle.
- Les données HORAIRES agrégées quotidiennement ne sont pas simplement équivalentes aux donnée QUOTIDIENNES car les paramètres ne sont pas strictement les mêmes. Certains paramètres horaires n'existent pas en quotidiens (ex. DRR1 durée des précipitations (en mn/heure)), et inversement

Auteur: https://github.com/loicduffar

##### 1) Lecture après téléchargement des archives GZ et décompression des fichiers CSV LATEST (dernière période disponible)

- Définir les chemins d'enregistrement pour les archives GZ et pour les fichier CSV décompressés
- Définir les postes à interroger par leur code à 8 chiffres (voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922)
- Définir UNE FOIS POUR TOUTES les urls de téléchargement du fichier LATEST pour chaque département
- NB: Les paramètres et la période à extraire sont définis dans la cellule 2 (pour pouvoir les modifier sans relire les fichiers)
- On peut éviter de télécharger-décompresser les archives GZ si les fichiers CSV sont déjà présents en local

Les archives LATEST sont automatiquement téléchargées et décompressées et les fichiers CSV sont lus

In [20]:
############ Auteur: L. Duffar ###########
############ Décembre 2023 ###########
# python 3.8.12
# Télécharge les archives GZ & et décompresse les fichiers CSV LATEST (DERNIERE PERIODE DISPONIBLE pour chaque département)
import os
import requests
import pandas as pd
import numpy as np
import datetime
import time
import sys
import gzip

# ================ Personalisation ====================
# Chemin d'enregistrement des archives gz et des fichiers CSV décompressés
folder_gz= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\H"
folder_csv= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\H"

# Numéro des Postes météo souhaités (chaine de 8 caractères) - voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922
# LES DEPARTEMENTS CORRESPONDANTS DOIVENT ËTRE PRESENTS DANS LA LISTE DES URLS (voir plus loin ======= Initialisation ==========)
# Attention: ne pas inclure des départements multiples car les fichiers horaires sont lourds à lire
postes= [
        # '04088001', '04039001', '04230001',                  # Forcalquier, Castellane, Valensole \
        # '05046001',                                           # Embrun \
        # '06088001', '06029001',                               # Nice, Cannes \
        '13001009', '13111002', '13103001','13055001', '13005003', '13028001', '13030001',  # Aix en Provence, Vauvenargue, Salon, Marseille, AUBAGNE, LA CIOTAT, CUGES-LES-PINS	\
        '13031002', '13062002', '13074003', '13091002', '13110003',   # LA DESTROUSSE, MIMET, PEYROLLES-EN-PROVENCE, SAINT-CANNAT, TRETS, \
        # '83031001', '83061001', '83137001',                   # Le Luc, Fréjus, Toulon \
        # '84003002', '84009002',                               # # Apt-Viton, Bastide des Jourdans \
        ]                               

# Choisir d'éviter si besoin le téléchargemnt/décompression
# non recommandé, sauf pour test ou à moins que le dernier téléchargement date du jour même
download= False

# ================ Extrait du fichier data CSV ============= (les colonnes sont en réalité séparées par des points-virgules)
# NUM_POSTE	NOM_USUEL	LAT	LON	ALTI	AAAAMMJJHH	RR1	QRR1	DRR1	QDRR1	FF	QFF	DD	QDD	FXY	QFXY
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	2022010100	0.2	1			0	1	0	1	0.8	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	2022010101	0	1			0.5	1	30	1	1.4	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	2022010102	0	1			0.5	1	60	1	0.9	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	2022010103	0	1			0	1	0	1	1	1
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	2022010104	0	1			0.7	1	160	1	0.7	1

# ================ Extrait du fichier CSV Descriptif de quelques paramètres les plus souvent utiles (précipitations et températures) =============(les colonnes sont en réalité séparées par ":")
# NUM_POSTE   	 numéro Météo-France du poste sur 8 chiffres
# NOM_USUEL   	 nom usuel du poste
# LAT         	 latitude, négative au sud (en degrés et millionièmes de degré)
# LON         	 longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI        	 altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMMJJHH  	 date de la mesure (année mois jour heure)
# RR1         	 quantité de précipitation tombée en 1 heure (en mm et 1/10 mm)
# DRR1        	 durée des précipitations (en mn)
# FF          	 force du vent moyenné sur 10 mn, mesurée à 10 m (en m/s et 1/10)
# DD          	 direction de FF (en rose de 360)
# FXY         	 valeur maximale de FF dans l’heure (en m/s et 1/10)
# DXY         	 direction de FXY (rose de 360)
# HXY         	 heure de FXY (hhmm)
# FXI         	 force maximale du vent instantané dans l’heure, mesurée à 10 m (en m/s et 1/10)
# DXI         	 direction de FXI (en rose de 360)
# HXI         	 heure de FXI (hhmm)
# FF2         	 force du vent moyenné sur 10 mn, mesurée à 2 m (en m/s et 1/10)
# DD2         	 direction de FF2 (en rose de 360)
# FXI2        	 force maximale du vent instantané dans l’heure, mesurée à 2 m (en m/s et 1/10)
# DXI2        	 direction de FXI2 (en rose de 360)
# HXI2        	 heure de FXI2 (hhmm)
# FXI3S       	 force maximale du vent moyenné sur 3 secondes dans l’heure (en m/s et 1/10)
# DXI3S       	 direction de FXI3S (en rose de 360)
# HXI3S       	 heure de FXI3S (hhmm)
# T           	 température sous abri instantanée (en °C et 1/10)
# TD          	 température du point de rosée (en °C et 1/10)
# TN          	 température minimale sous abri dans l’heure (en °C et 1/10)
# HTN         	 heure de TN (hhmm)
# TX          	 température maximale sous abri dans l’heure (en °C et 1/10)
# HTX         	 heure de TX (hhmm)

# ================ Initialisation ====================
now= datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M"), '...Patientez, les données horaires sont lourdes à lire...')
# Structure du nom des fichiers de données QUOTIDIENNES (1 fichier par département dont le numéro sera ajouté automatiquement au début et à la fin du template ci-dessous)
template_start= 'H_' # début du nom du fichier
# Fin du nom dépendant de l'année en cours
template_end= '_latest-' + str(now.year-1) + '-' + str(now.year) + '.csv'

# ---------------- urls de téléchargement des archives PAR DEPARTEMENT des dernières données depuis janvier de l'année précédente https://meteo.data.gouv.fr/ (Ajouter d'autres départements si besoin)
url_liste_postes= "https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922"
url_desc_h= "https://www.data.gouv.fr/fr/datasets/r/5d0f9af9-149b-463a-9472-445dafb698d9"
url_desc_q= "https://www.data.gouv.fr/fr/datasets/r/6a8df7e9-45ff-445d-9260-6c65475dda86"

urls= dict()
urls['04']= "https://www.data.gouv.fr/fr/datasets/r/7d9c4e73-68b8-4e93-97a6-0f94feb21e50" 
urls['05']= 'https://www.data.gouv.fr/fr/datasets/r/053768ac-e0ce-4305-8d4a-c6d043ed5525'
urls['06']= 'https://www.data.gouv.fr/fr/datasets/r/1eb49075-5855-4390-a23f-0ecae7113fbd'
urls['13']= 'https://www.data.gouv.fr/fr/datasets/r/ab1273f1-81a5-41f6-980f-04981d7d7925'
urls['83']= 'https://www.data.gouv.fr/fr/datasets/r/9e6bec25-d7a7-4d25-9b5f-f06e4b935b96'
urls['84']= 'https://www.data.gouv.fr/fr/datasets/r/48bba355-ddd9-4265-87a2-e471a9d8703c'

# ---------------- Définition des fonctions
def convert_to_date(chaine):
    return pd.to_datetime(str(chaine), format='%Y%m%d%H', errors='coerce')

def download_file(url, filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    print('Téléchargement: ', file)
    response = requests.get(url)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
    else:
        print("Fichier d'archive non présent à l'url habituelle: ", file)

def decompress_gz(filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    
    if os.path.exists(file):
        with gzip.open(file, 'rb') as f_in:
            file= os.path.join(folder_csv, filename) 
            print('Décompression', file)
            with open(file, 'wb') as f_out:
                f_out.write(f_in.read())
    else:
        print("Fichier d'archive non trouvé: ", file)

def read_csv(filename):
    file= os.path.join(folder_csv, filename)
    if os.path.exists(file):
        print('Lecture: ', file)
        df= pd.read_csv(file, header=0, sep=";", dtype={"NUM_POSTE":str, 'AAAAMMJJHH':str}, parse_dates=['AAAAMMJJHH'], date_parser= convert_to_date)
    else:
        print("Fichier CSV non trouvé: ", file)
    return df

# ================ Traitement ====================
# Lit en ligne le fichier "fin" "H_descriptif_champs.csv" de description des champs
# définit un dataframe pandas avec les 2 colonnes "param" et "name_long" pour la description des champs 
df_desc_f = pd.read_csv(url_desc_h, sep=":", header= None, index_col=0, names= ["param", "name_long", 'complement'], dtype={"param":str, "name_long":str, 'complement':str}, encoding= 'utf-8')
df_desc_f.index= df_desc_f.index.str.strip()
df_desc_f['name_long']= df_desc_f['name_long'].str.strip()

# Lit en ligne le fichier "Q_descriptif_champs_RR-T-Vent.csv" de description des champs
# définit un dataframe pandas avec les 2 colonnes "param" et "name_long" pour la description des champs 
df_desc_bis = pd.read_csv(url_desc_q, sep=":", header= None, index_col=0, names= ["param", "name_long", 'complement'], dtype={"param":str, "name_long":str, 'complement':str}, encoding= 'utf-8')
df_desc_bis.index= df_desc_bis.index.str.strip()
df_desc_bis['name_long']= df_desc_bis['name_long'].str.strip()

# fait la liste des départements concernés et garde les 2 premiers chiffres uniques
departements = []
for poste in postes:
    departements.append(poste[:2])
# On ne garde que départements uniques
departements = list(set(departements)) # set() ignore automatiquement les doublons (ce qui évite de passer par array numpy pour utiliser la fonction np.unique() )
# On trie la liste
departements.sort()
# On affiche la liste
print('Départements concernés: ', departements)

# Téléchargement/décompression/lecture des fichiers dans une boucle sur les départements (urls tirées du dictionnaire 'urls')
for i, departement in enumerate(departements):
    # On récupère l'url
    url = urls[departement]
    # Formation du nom du fichier à partir du template et du numéro de département
    filename = f"{template_start}{departement}{template_end}"
    if download:
        download_file(url, filename)
        decompress_gz(filename)
    # Lecture du fichier CSV dans un dataframe pandas
    df_departement= read_csv(filename)
    if i== 0:
        # initialisation du dataframe final df avec les données du premier département        
        df= df_departement
    else:
        df= pd.concat([df, df_departement])        

# supprime les espaces avant et après le numéro de poste
df['NUM_POSTE'] = df['NUM_POSTE'].str.strip()
# renome la colonne "AAAMMJJ" en "DATE"
df.rename(columns={'AAAAMMJJHH':'DATE'}, inplace=True)
# supprime les espaces dans les noms de colonnes
df.columns= df.columns.str.strip()

# affiche la durée du traitement depuis now
now2= datetime.datetime.now()
print('Fin: ', now2.strftime("%Y-%m-%d %H:%M"))
print('Durée du traitement: ', now2 - now)

display(df)

2023-12-26 21:42 ...Patientez, les données horaires sont lourdes à lire...
Départements concernés:  ['13']
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\H\H_13_latest-2022-2023.csv
Fin:  2023-12-26 21:43
Durée du traitement:  0:01:46.641671


,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,DATE,RR1,QRR1,DRR1,QDRR1,...,INS,QINS,INS2,QINS2,TLAGON,QTLAGON,TVEGETAUX,QTVEGETAUX,ECOULEMENT,QECOULEMENT
0,13001009,AIX EN PROVENCE,43.5295,5.4245,173,2022-01-01 00:00:00,0.2,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13001009,AIX EN PROVENCE,43.5295,5.4245,173,2022-01-01 01:00:00,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13001009,AIX EN PROVENCE,43.5295,5.4245,173,2022-01-01 02:00:00,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13001009,AIX EN PROVENCE,43.5295,5.4245,173,2022-01-01 03:00:00,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13001009,AIX EN PROVENCE,43.5295,5.4245,173,2022-01-01 04:00:00,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364886,13111002,VAUVENARGUES,43.5510,5.6800,565,2023-12-25 23:00:00,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364887,13111002,VAUVENARGUES,43.5510,5.6800,565,2023-12-26 00:00:00,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364888,13111002,VAUVENARGUES,43.5510,5.6800,565,2023-12-26 01:00:00,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364889,13111002,VAUVENARGUES,43.5510,5.6800,565,2023-12-26 02:00:00,0.0,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 2) Affichage des données horaires & Graphique des précipitations 

- Définir comme suit la période d'extraction : (dans tous les cas le graphique de 7 JOURS peut se déplacer à l'intérieur de la période d'extraction)
    - Laissez le script la fixer automatiquement (depuis le 1er jour du mois précédent la fin des données)
    - fixer la date de début et de fin
    - ou None pour maximiser la période totale (pénalisant pour le poids de données)
- Définir le ou les paramètres à extraire (au minimum les précipitations RR car le graphique est uniquement prévu pour cela)
- NB: Tous les paramètres sont des chiffres, y compris ceux donnant une heure (ex. HTN= 738 -> Heure du minimum de température= 07:38)

In [21]:
# trace un graphique plotly avec la colonne 'DATE' en abscisse et la colonne 'RR' en ordonnée
# chaque poste de la colonnee 'NUM_POSTE' est représenté dans un subplot séparé
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
import calendar
import numpy as np

#-------------- Personalisation du graphique
# Paramètres à représenter: 'RR1' obligatoire en premier, pour le graphique dont la présentation actuelle est spécialement adaptée aux précipitations
param_list= ['RR1', 'T']
agregat_list= ['sum', 'mean'] # 'sum' pour la somme (précipitations), 'mean' pour la moyenne (température moyenne), max (vent), min (Tempéraure min)
# TODO: prendre en compte le Nombre maxi de NaN dans les données pour l'agrégation journalière
# agregat_max_nan_list= [0, 5]

# Période A EXTRAIRE (None pour la période par défaut)
start_date= None
start_date= datetime.datetime(2023, 12, 8, 0)
# Calculer start_date au 1er jour du mois précédent
if now.month == 1:  # Si aujourd'hui est en janvier
    start_date = datetime.datetime(now.year - 1, 12, 1)
else:  # Pour tous les autres cas
    start_date = datetime.datetime(now.year, now.month-1, 1)

end_date= None
end_date= datetime.datetime(2023, 12, 12)
end_date= df['DATE'].max()

#-------------- initialisation des paramètres du graphique
param_RR= 'RR1'
# Période par défaut d'extraction (Si start_date= None ou end_date= None) : maximisation de l'extraction
if start_date is None:
    start_date= df['DATE'].max() - datetime.timedelta(days= 90)
if end_date is None:
    end_date= df['DATE'].max()

# Période du GRAPHIQUE (plus réduite que celle des données mais laissant la possibilité de faire défiler le graphique)
start_graph= datetime.datetime(end_date.year, end_date.month, end_date.day+1, 0) - datetime.timedelta(days= 8)
end_graph= end_date

# filtre le dataframe sur les postes choisis
df_postes= df[df['NUM_POSTE'].isin(postes)]
# filtre le dataframe sur la période souhaitée
df_postes= df_postes[(df_postes['DATE'] >= start_date) & (df_postes['DATE'] <= end_date)]

#  Couleurs des courbes
color_sequence_bar = plotly.colors.qualitative.Light24 #Dark24
color_sequence_scatter = plotly.colors.qualitative.Alphabet

# Axe des ordonnées secondaire (cumulé)
specs= [[{'secondary_y': True}]]*len(postes)

#-------------- Graphique
fig = make_subplots(rows= len(postes), cols= 1,# shared_xaxes= True, # shared_xaxes= True inutile grâce à  matches='x' dans update_xaxes qui fait la même chose sans supprimer la graduation des dates des subplots 
                    subplot_titles= [f"{poste} - {df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]}" for poste in postes], specs= specs)

for i, poste in enumerate(postes):
    df_poste = df_postes[df_postes['NUM_POSTE'] == poste]
    fig.add_trace(go.Bar(x= df_poste['DATE'], y= df_poste[param_RR], 
                          # name= poste
                         name= df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0],
                         marker= dict(color=color_sequence_bar[i % len(color_sequence_bar)],  # Utilisez la couleur correspondante de la séquence
                                    line= dict(color='rgb(100,100,100)', width=1),  # Ligne de contour des barres
                                    ),
                        # hovertemplate = 
                            # '<i>Date</i>: %{x}' +
                            # '<br><b>RR1</b>: %{y}<br>',
                            # text = df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]                                    
                         ),
                row= i+1, col= 1
                )
    fig.add_trace(go.Scatter(x= df_poste['DATE'], y= df_poste[param_RR].cumsum(), 
                             name= f"Cumulé {poste}", yaxis= "y2",
                             line= dict(color= 'gray'
                            #  line= dict(color= color_sequence_scatter[i % len(color_sequence_scatter)]
                                        ), # sinon l'incrémentation dans la séquence de couleurs est de +2 à cause de la trace Bar
                    # hovertemplate = 
                    #         '<br><b>Cumulé</b>: %{y}<br>'                                      
                             ), 
                 row=i+1, col=1, secondary_y= True,
                 )

fig.update_layout(title_text= 'meteo.data.gouv.fr - ' + "Précipitations horaires (mm) - " + now.strftime("%Y-%m-%d") , title_x= 0.5, 
                  height= 133.*len(postes), width= 1000,
                  hovermode= 'x unified', hoverlabel= dict(bgcolor='rgba(255,255,255,0.6)'),
                    )

# Survol par la souris : ligne verticale à travers tous les subplots matérialisant l'abscisse 
xlast= 'x' + str(len(postes))
# fig.update_traces(xaxis= xlast)  # xlast est le nom de l'axe des abscisses du dernier subplot (empêche la graduation des dates de tous les subplots)

# Définition générale des axes
max_value= df_postes[param_RR].max()
max_value_cum= df_postes.groupby(['NUM_POSTE']).sum()[param_RR].max()
fig.update_yaxes(secondary_y= False, range= [0, max_value])  # Axe principale: Replace max_value with the desired maximum coordinate
fig.update_yaxes(secondary_y= True, title_text= "Cumulé", range= [0, max_value_cum]) # Axe secondaire: Replace max_value with the desired maximum coordinate

# GRADUATION de l'axe des DATES ........................................
if end_graph - start_graph > datetime.timedelta(days= 7):
    tickformat= '%d %Hh\n%b %y'
    dtick= 'D2'      # pour une raison inconnue,'D2' ne marche pas
else: # Moins
    tickformat= '%d %Hh\n%b n%Y'
    dtick= 'D' # pour une raison inconnue,'D10' ne marche pas du tout et le nombre de milisecondes ne marche qu'après la 2ème graduation et )
# ............................... Premiers subplots
for i in range(1, len(postes) + 1):
    fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1, matches='x')
# ............................... Dernier subplot
fig.update_xaxes(title_text= 'Heure UTC',tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1, matches='x', range=[start_graph, end_graph])

# GRILLE.................................................................
# .................. verticales 
# .............................. Premiers subplots INUTILE ?
for i in range(1, len(postes) + 1):
    fig.update_xaxes(showgrid= True, tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1)
# .............................. Dernier subplot TOUS ?
fig.update_xaxes(showgrid= True, tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1)
# .................. horizontales
# ............................... Axe secondaire
fig.update_yaxes(secondary_y=True, showgrid=False) # pas de grille

#-------------- sauvegarde le graphique
file_graph= "meteo.data HOR"
fig.write_html(os.path.join(folder_csv, file_graph + template_end[:-4] + ".html"))
fig.write_image(os.path.join(folder_csv, file_graph + template_end[:-4] + ".png"))

#-------------- Données fines (horaires) pivotées avec les postes en colonnes (pour la présentation ainsi que pour le calcul d'es 'agrégation quotidienne) 
#               pour les postes et les paramètres SELECTIONNES, SUR la durée COMPLETE
for i, par in enumerate(param_list):
    if i==0:
        df_f = df[df['NUM_POSTE'].isin(postes)].pivot(index='DATE', columns='NOM_USUEL', values= param_list[i]) \
            .assign(parametre= par)
    else:
        # append des lignes à df_f
        df_f = pd.concat([df_f,
                        df[df['NUM_POSTE'].isin(postes)].pivot(index='DATE', columns='NOM_USUEL', values= param_list[i]) \
                        .assign(parametre= par)]
                        )
df_f= df_f.reset_index().set_index(['parametre', 'DATE'], drop= True)

#-------------- Agrégation quotidienne par cumul ou moyenne selon le paramètre, en affectant NaN aux jours incomplets
#               pour les postes et les paramètres SELECTIONNES, SUR la durée COMPLETE
# TODO: préciser le test pour un nombre maxi de NaN dans les données agregat_max_nan_list
def sum_or_nan(s):
    return s.sum() if s.count() == s.size else np.nan
def mean_or_nan(s):
    return s.mean() if s.count() == s.size else np.nan
def max_or_nan(s):
    return s.max() if s.count() == s.size else np.nan
def min_or_nan(s):
    return s.min() if s.count() == s.size else np.nan

for i, par in enumerate(param_list):
    if i==0:
        if agregat_list[0] == 'sum':
            df_agr= df_f.loc[par].resample('D').apply(sum_or_nan)
        elif agregat_list[0] == 'mean':
            df_agr= df_f.loc[par].resample('D').apply(mean_or_nan)
        elif agregat_list[0] == 'max':
            df_agr= df_f.loc[par].resample('D').apply(max_or_nan)
        df_agr= df_agr.reset_index().set_index([[par for j in range(df_agr.shape[0])], 'DATE'])
    else:
        if agregat_list[i] == 'sum':
            df_temp= df_f.loc[par].resample('D').apply(sum_or_nan)
        elif agregat_list[i] == 'mean':
            df_temp= df_f.loc[par].resample('D').apply(mean_or_nan)
        elif agregat_list[i] == 'max':
            df_temp= df_f.loc[par].resample('D').apply(max_or_nan)
        df_agr= pd.concat([df_agr, 
                        df_temp.reset_index().set_index([[par for j in range(df_temp.shape[0])], 'DATE'])]
                        )
# arrondi df_agr à 1 chiffre après la virgule
df_agr= df_agr.round(1)

#-------------- Affichage
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
# imprimer un avertissement si la dernière date du dataframe est antérieure au dernier jour du mois en cours
# calculer la date du dernier jour du mois en cours en tenant compte des jours à 28, 29 et 30 jours
last_date= datetime.datetime(now.year, now.month, calendar.monthrange(now.year, now.month)[1])
titre_agr= 'Agrégation quotidienne pour la période récente et les paramètres choisis: '
if np.datetime64(df['DATE'].values[-1]) < np.datetime64(last_date):
    titre_agr= titre_agr + 'Attention le cumul du dernier jour est incomplet à cet instant'
print(titre_agr)

for i, par in enumerate(param_list):
    # teste si df_desc_bis.loc[par] existe
    if par in df_desc_bis.index:
        titre= '- ' + par + ': ' + df_desc_bis.loc[par]['name_long']
    else:
        titre= '- ' + par + ': ' + 'agrégation quotidienne "' + agregat_list[i] + '" de "' + df_desc_f.loc[par]['name_long'] + '"'
    print(titre)
display(df_agr[(df_agr.index.get_level_values(1) >= start_graph) & (df_agr.index.get_level_values(1) <= end_graph)])

fig.show()

titre_f= 'Données horaires pour les paramètres et la période choisis: '
print(titre_f)
for i, par in enumerate(param_list):
    titre= '- ' + par + ': ' + df_desc_f.loc[par]['name_long']
    print(titre)
    display(df_f.loc[par][(df_f.loc[par].index >= start_date) & (df_f.loc[par].index <= end_date)])

2023-12-26 21:45
Agrégation quotidienne pour la période récente et les paramètres choisis: Attention le cumul du dernier jour est incomplet à cet instant
- RR1: agrégation quotidienne "sum" de "quantité de précipitation tombée en 1 heure (en mm et 1/10 mm)"
- T: agrégation quotidienne "mean" de "température sous abri instantanée (en °C et 1/10)"


NOM_USUEL       AIX EN PROVENCE  AUBAGNE  BEC DE L AIGLE  CUGES-LES-PINS  \
    DATE                                                                   
RR1 2023-12-19              0.2      0.2             0.0             0.0   
    2023-12-20              0.0      0.2             0.0             0.2   
    2023-12-21              0.0      0.0             0.0             0.0   
    2023-12-22              0.0      0.0             0.0             0.0   
    2023-12-23              0.0      0.0             0.0             0.0   
    2023-12-24              0.2      0.0             0.0             0.0   
    2023-12-25              0.2      0.2             0.0             0.0   
    2023-12-26              0.0      0.0             0.0             NaN   
T   2023-12-19              5.5      5.3            12.6             3.3   
    2023-12-20              6.8      7.7             8.7             7.3   
    2023-12-21             10.6     11.5            10.1            11.7   
    2023-12-22             12.8     13.9            12.3            13.5   
    2023-12-23             10.3     11.6            10.9            12.8   
    2023-12-24              6.6      9.0            12.4            10.5   
    2023-12-25              5.1      5.4            12.5             4.5   
    2023-12-26              0.4      0.1             9.8             NaN   

NOM_USUEL       LA DESTROUSSE_SAPC  MARSEILLE-OBS  MIMET  \
    DATE                                                   
RR1 2023-12-19                 0.0            0.0    0.0   
    2023-12-20                 0.2            0.0    0.0   
    2023-12-21                 0.0            0.0    0.0   
    2023-12-22                 0.0            0.0    0.0   
    2023-12-23                 0.0            0.0    0.0   
    2023-12-24                 0.0            0.0    0.0   
    2023-12-25                 0.0            0.0    0.0   
    2023-12-26                 0.0            0.0    0.0   
T   2023-12-19                 4.1            9.6    7.3   
    2023-12-20                 6.4            9.5    6.0   
    2023-12-21                 9.8           11.1    8.3   
    2023-12-22                12.6           13.5   10.4   
    2023-12-23                10.6           12.2    8.5   
    2023-12-24                 8.2           12.1    8.9   
    2023-12-25                 4.5           10.1    9.9   
    2023-12-26                -1.6            6.4    5.1   

NOM_USUEL       PEYROLLES EN PROVENCE  SALON DE PROVENCE  ST CANNAT  TRETS  \
    DATE                                                                     
RR1 2023-12-19                    1.6                0.2        0.0    0.0   
    2023-12-20                    0.4                0.0        0.0    0.2   
    2023-12-21                    0.0                0.0        0.0    0.0   
    2023-12-22                    0.2                0.0        0.0    0.0   
    2023-12-23                    0.2                0.0        0.0    0.0   
    2023-12-24                    0.0                0.0        0.0    0.0   
    2023-12-25                    0.4                0.0        0.0    0.0   
    2023-12-26                    0.0                0.0        0.0    0.0   
T   2023-12-19                    5.0                4.6        7.0    5.7   
    2023-12-20                    5.1                9.6        7.3    6.1   
    2023-12-21                    9.5               10.7        9.4    9.6   
    2023-12-22                   13.3               13.1       11.9   12.4   
    2023-12-23                   10.5               11.7        9.9   10.4   
    2023-12-24                    6.1               12.5       10.2    9.1   
    2023-12-25                    4.4                8.0        5.4    5.3   
    2023-12-26                    0.4               -0.6        1.9    0.0   

NOM_USUEL       VAUVENARGUES  
    DATE                      
RR1 2023-12-19           0.0  
    2023-12-20           0.0  
    2023-12-21       

Données horaires pour les paramètres et la période choisis: 
- RR1: quantité de précipitation tombée en 1 heure (en mm et 1/10 mm)


NOM_USUEL,AIX EN PROVENCE,AUBAGNE,BEC DE L AIGLE,CUGES-LES-PINS,LA DESTROUSSE_SAPC,MARSEILLE-OBS,MIMET,PEYROLLES EN PROVENCE,SALON DE PROVENCE,ST CANNAT,TRETS,VAUVENARGUES
DATE,,,,,,,,,,,,
2023-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-01 04:00:00,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-25 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-26 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-12-26 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- T: température sous abri instantanée (en °C et 1/10)


NOM_USUEL,AIX EN PROVENCE,AUBAGNE,BEC DE L AIGLE,CUGES-LES-PINS,LA DESTROUSSE_SAPC,MARSEILLE-OBS,MIMET,PEYROLLES EN PROVENCE,SALON DE PROVENCE,ST CANNAT,TRETS,VAUVENARGUES
DATE,,,,,,,,,,,,
2023-11-01 00:00:00,9.2,9.5,15.0,9.1,8.0,13.5,11.7,7.6,9.8,8.3,9.1,10.7
2023-11-01 01:00:00,9.3,9.2,15.0,8.6,7.8,13.3,10.9,8.5,9.7,8.5,8.8,9.9
2023-11-01 02:00:00,9.2,9.2,15.1,8.5,7.7,13.4,10.5,8.6,10.1,8.5,8.4,8.2
2023-11-01 03:00:00,9.3,9.5,15.1,8.8,7.7,13.4,11.2,8.5,10.6,8.7,9.2,7.8
2023-11-01 04:00:00,9.5,9.6,15.2,8.5,7.6,13.5,10.4,8.3,11.1,9.4,8.9,7.6
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-25 23:00:00,1.1,0.7,10.4,-0.3,-0.8,7.1,5.7,0.7,2.0,2.0,1.1,3.4
2023-12-26 00:00:00,1.0,0.2,10.3,-1.0,-1.2,6.9,5.7,-0.2,0.3,2.2,0.4,2.6
2023-12-26 01:00:00,0.3,0.2,9.5,-1.5,-1.5,6.4,5.2,-0.3,-0.5,2.1,-0.2,3.0


##### 3) Affichage de l'agrégation quotidienne & graphique des précipitations mensuelles

Définir au préalable comme suit les dates de début et de fin du graphique:
- "None" pour prendre le début et la fin du fichier LATEST (premier jour de l'année précédente et veille du téléchargement)
- ou définir la date de début et de fin

NB: Par défault les paramètres sont les mêmes que ceux choisis dans la cellule 2

In [22]:
# trace un graphique chronologique des précipitations mensuelles
# chaque poste de la colonne 'NUM_POSTE' est représenté dans un subplot séparé
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

#-------------- Personalisation du graphique 
# Paramètre à représenter (la présentation actuelle est spécialement adaptée aux précipitations RR)
# param= 'RR'

# Période à représenter (None pour le début ou la fin du fichier par défaut)
# start_date= None
# start_date= datetime.datetime(2022, 5, 1)

# end_date= None
# end_date= datetime.datetime(now.year, now.month, now.day)

#-------------- initialisation des paramètres du graphique
# à partir de df_postes, génère un dataframe indexé sur 'NOM_USUEL' avec une colonne 'NUM_POSTE' et
df_desc_postes= df_postes[['NUM_POSTE', 'NOM_USUEL']].drop_duplicates().set_index('NOM_USUEL')

# réorganise le dataframe df_agr (intermédiaire ayant permis le cumul mensuel poste par poste) avec les postes en lignes et une seule colonne 'RR'
df_p= df_agr.loc[param_RR].stack().reset_index(name= param_RR)
# ajoute à df_p une colonne 'NUM_POSTE' avec le numéro du poste
df_p= df_p.join(df_desc_postes, on='NOM_USUEL')

# Période par défaut if start_date= None ou  end_date= None
# if start_date is None:
#     start_date= df_p.reset_index()['DATE'].min()
# if end_date is None:
#     end_date= df_p.reset_index()['DATE'].max()

# filtre le dataframe sur les postes choisis 
df_p= df_p[df_p['NUM_POSTE'].isin(postes)]
# filtre le dataframe sur la période souhaitée
df_p= df_p[(df_p['DATE'] >= start_date) & (df_p['DATE'] <= end_date)]

#  Couleurs des courbes
color_sequence_bar = plotly.colors.qualitative.Set3
# color_sequence_scatter = plotly.colors.qualitative.Dark24

# Axe des ordonnées secondaire (cumulé)
specs= [[{'secondary_y': True}]]*len(postes)

#-------------- Graphique
fig = make_subplots(rows= len(postes), cols= 1,# shared_xaxes= True, # shared_xaxes= True inutile grâce à  matches='x' dans update_xaxes qui fait la même chose sans supprimer la graduation des dates des subplots 
                    subplot_titles= [f"{poste} - {df_p[df_p['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]}" for poste in postes], specs= specs)

for i, poste in enumerate(postes):
    df_poste = df_p[df_p['NUM_POSTE'] == poste]
    fig.add_trace(go.Bar(x= df_poste['DATE'], y= df_poste[param_RR], 
                          # name= poste
                         name= df_p[df_p['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0],
                         marker= dict(color=color_sequence_bar[i % len(color_sequence_bar)]),  # Utilisez la couleur correspondante de la séquence
                         ),
                row= i+1, col= 1)
    fig.add_trace(go.Scatter(x= df_poste['DATE'], y= df_poste[param_RR].cumsum(), 
                             name= f"Cumulé {poste}", yaxis= "y2",
                             line= dict(color= 'gray'),
                            #  line= dict(color= color_sequence_scatter[i % len(color_sequence_scatter)]), # sinon l'incrémentation dans la séquence de couleurs est de +2 à cause de la trace Bar
                             mode='lines' 
                             ), 
                 row=i+1, col=1, secondary_y= True)

fig.update_layout(title_text= 'meteo.data.gouv.fr - ' + "Précipitations quotidiennes (mm) - " + now.strftime("%Y-%m-%d") , title_x= 0.5, 
                  height= 133.*len(postes), width= 1000,
                  hovermode= 'x unified', hoverlabel= dict(bgcolor='rgba(255,255,255,0.6)'),
                  )

# Survol par la souris : ligne verticale à travers tous les subplots matérialisant l'abscisse 
xlast= 'x' + str(len(postes))
# fig.update_traces(xaxis= xlast)  # xlast est le nom de l'axe des abscisses du dernier subplot

# Définition générale des axes
max_value= df_p[param_RR].max()
max_value_cum= df_p.groupby(['NUM_POSTE']).sum()[param_RR].max()
fig.update_yaxes(secondary_y= False, range= [0, max_value])  # Axe principale: Replace max_value with the desired maximum coordinate
fig.update_yaxes(secondary_y= True, title_text= "Cumulé", range= [0, max_value_cum]) # Axe secondaire: Replace max_value with the desired maximum coordinate

# Graduations de l'axe des DATES ...............................................................
if end_date - start_date > datetime.timedelta(days= 365): # plus d'un an
    tickformat= '%d-%b\n%Y'
    dtick= 'M2'
    showgrid_x= True
else: # Moins
    tickformat= '%d %b\n%Y'
    dtick= 'M1' # (pour une raison inconnue,'D10' ne marche pas du tout et le nombre de milisecondes ne marche qu'après la 2ème graduation et )
    showgrid_x= False
    
# ...............................Premiers subplots
# ................................................ en fonction de la longueur de la période
for i in range(1, len(postes) + 1):
    fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1, matches='x')
# ...............................Dernier subplot
# ................................................ en fonction de la longueur de la période
fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1, matches='x')
#                                                SANS tenir compte de la longueur de la période
# fig.update_xaxes(tickformat= '%b\n%Y', tickmode= 'linear', dtick= 'M1', row=len(postes), col=1, matches='x')

# GRILLE .............................................................................
# .................. verticales...
fig.update_xaxes(showgrid= showgrid_x, gridwidth= 1, gridcolor= 'white')
# .................. horizontales
#                                Axe secondaire:
fig.update_yaxes(secondary_y=True, showgrid=False) # pas de grille

#-------------- sauvegarde le graphique
file_graph_agr= "meteo.data QUOT"
fig.write_html(os.path.join(folder_csv, file_graph_agr + template_end[:-4] + ".html"))
# sauvegarde le graphique dans un fichier png
fig.write_image(os.path.join(folder_csv, file_graph_agr + template_end[:-4] + ".png"))

#-------------- Affichage
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
fig.show()
# afficher RR pour df_p avec les postes en colonnes et les dates en index
print('Agrégation quotidienne des précipitations LATEST (mm)')
display(df_p.pivot(index= 'DATE', columns= 'NOM_USUEL', values= param_RR))

2023-12-26 21:47


Agrégation quotidienne des précipitations LATEST (mm)


NOM_USUEL,AIX EN PROVENCE,AUBAGNE,BEC DE L AIGLE,CUGES-LES-PINS,LA DESTROUSSE_SAPC,MARSEILLE-OBS,MIMET,PEYROLLES EN PROVENCE,SALON DE PROVENCE,ST CANNAT,TRETS,VAUVENARGUES
DATE,,,,,,,,,,,,
2023-11-01,0.2,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.2
2023-11-02,4.2,6.2,4.4,6.4,6.8,4.8,6.8,5.6,3.0,5.2,6.6,10.8
2023-11-03,0.6,0.4,1.0,0.8,0.0,1.8,0.0,0.4,0.4,0.8,0.0,0.2
2023-11-04,1.4,2.0,1.2,4.8,3.8,1.2,0.6,1.8,0.4,2.4,1.4,5.8
2023-11-05,0.4,0.2,0.0,0.2,0.2,0.0,0.0,0.8,0.0,0.4,0.2,0.6
2023-11-06,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.2,1.4,0.4,0.0,0.0
2023-11-07,0.2,0.4,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0
2023-11-08,0.2,0.2,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.4,0.0,0.0
2023-11-09,7.0,10.6,17.1,11.0,8.6,8.0,7.8,3.4,15.4,9.8,6.6,8.4


##### 4) Sauvegarde les données intégrales dans un fichier Excel (tous les paramètres pour tous les postes des départements concernés)
Le fichier est sauvegardé dans le même dossier local que les fichiers CSV décompressés

In [23]:
# Sauvegarde toutes les informations dans un fichier Excel
print(now.strftime("%Y-%m-%d %H:%M"))
print('Sauvegarde en cours, patientez...')
file_excel= os.path.join(folder_csv, "meteo.data HOR"+ '_' + '-'.join(departements) + template_end[:-4] + ".xlsx")
writer = pd.ExcelWriter(file_excel, engine='xlsxwriter')
workbook  = writer.book
#======================== Sauvegarde la liste des postes dans le fichier excel
# Lecture en ligne du fichier JSON
req= requests.get(url_liste_postes)
if req.status_code == 200:
    data_json= req.json()
else:
    print('la requête a échoué avec le code : ', req.status_codes)
    sys.exit() # interrompt le script
df_liste_postes = pd.DataFrame(data_json['features']).T
# Ajout de colonnes avec les champs de properties
df_liste_postes['lat'] = df_liste_postes['geometry'].apply(lambda x: x['coordinates'][1]).astype(float)
df_liste_postes['lon'] = df_liste_postes['geometry'].apply(lambda x: x['coordinates'][0]).astype(float)
df_liste_postes['nom_usuel']= df_liste_postes['properties'].apply(lambda x: x['NOM_USUEL'].strip())
df_liste_postes['num_poste']= df_liste_postes['properties'].apply(lambda x: x['NUM_POSTE'].strip())
df_liste_postes['commune'] = df_liste_postes['properties'].apply(lambda x: x['COMMUNE'].strip())
df_liste_postes['ficheClimComplete'] = df_liste_postes['properties'].apply(lambda x: x['ficheClimComplete']).astype(float)
df_liste_postes['ficheClimReduite'] = df_liste_postes['properties'].apply(lambda x: x['ficheClimReduite']).astype(float)
df_liste_postes['alti'] = df_liste_postes['properties'].apply(lambda x: x['ALTI'])
# sauvegarde dans un fichier excel plus lisible que le JSON après suppression des colonnes type, geometry et properties
df_liste_postes.drop(['type', 'geometry', 'properties'], axis=1, inplace=True)
df_liste_postes.sort_values(by= ['num_poste'], inplace=True)
sheet= 'liste_postes'
df_liste_postes.to_excel(writer, sheet_name= sheet, index= False, startrow= 0)

#======================== Sauvegarde la liste des champs dans le fichier Excel
sheet= 'H_descriptif_champs_RR-T-Vent'
df_desc_f.to_excel(writer, sheet_name= sheet, index= True, startrow= 1)
worksheet = writer.sheets[sheet]
worksheet.write('A1', sheet + '.csv')

#======================== Sauvegarde les données au format original    DONNEES HORAIRES COMPLETES TROP VOLUMINEUSES         
sheet= 'format original'
# df.to_excel(writer, sheet_name= sheet, index=False, startrow= 2)
# worksheet = writer.sheets[sheet]
# worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
# worksheet.write('A2', 'Données LATEST horaires au format original, complètes pour tous les départements concernés: ')

#======================== Sauvegarde les données à un format plus lisible
n_param= len(param_list)
sheet= 'HOR Sélection'
df_f[(df_f.index.get_level_values(1) >= start_date) & (df_f.index.get_level_values(1) <= end_date)] \
    .to_excel(writer, sheet_name= sheet, index=True, startrow= n_param + 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', titre_f)
for i, par in enumerate(param_list):
    titre= '- ' + par + ': ' + df_desc_f.loc[par]['name_long']
    worksheet.write('A' + str(i+3), titre)

sheet= 'QUOT Sélection'
df_agr[(df_agr.index.get_level_values(1) >= start_date) & (df_agr.index.get_level_values(1) <= end_date)] \
    .to_excel(writer, sheet_name= sheet, index=True, startrow= n_param + 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', titre_agr)
for i, par in enumerate(param_list):
    if par in df_desc_bis.index:
        titre= '- ' + par + ': ' + df_desc_bis.loc[par]['name_long']
    else:
        titre= '- ' + par + ': ' + 'agrégation quotidienne "' + agregat_list[i] + '" de "' + df_desc_f.loc[par]['name_long'] + '"'
    worksheet.write('A' + str(i+3), titre)

# ajoute un onglet excel et enregistre une image du graphique
sheet4= 'graph'
worksheet = workbook.add_worksheet(sheet4)
worksheet.insert_image('A1', os.path.join(folder_csv, file_graph + template_end[:-4] + ".png"))
worksheet.insert_image('Q1', os.path.join(folder_csv, file_graph_agr + template_end[:-4] + ".png"))

writer.save()

print('Ficher Excel sauvegardé: ', file_excel)

2023-12-26 21:42
Sauvegarde en cours, patientez...
Ficher Excel sauvegardé:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\H\meteo.data HOR_13_latest-2022-2023.xlsx
